In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [3]:
def crawling():
    driver = webdriver.Chrome()

    # 일반 공지
    driver.get('https://www.dongguk.edu/article/GENERALNOTICES/list')

    time.sleep(2)

    items = driver.find_elements(By.CSS_SELECTOR, 'div.board_list li')
    url_list=[]

    for i in range(len(items)):
        # refresh 안해주면 하나만 크롤링하고 끝남
        items = driver.find_elements(By.CSS_SELECTOR, 'div.board_list li')
        try:
            num_span = items[i].find_element(By.XPATH, './/a/div[@class="mark"]/span[@class="num"]')
            num_span.click()
            time.sleep(3)
            
            # add code
            url = driver.current_url
            url_list.append(url)
            driver.back()
            time.sleep(2)
            
        except Exception as e:
            pass

    driver.quit()
    return url_list

url_list = crawling()

In [3]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# 이미지 저장 경로 설정
image_folder = os.path.join(os.getcwd(), "image")
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

def download_image(image_url, save_path):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            with open(save_path, "wb") as img_file:
                img_file.write(response.content)
            print(f"Image saved at {save_path}")
        else:
            print(f"Failed to download image from {image_url}")
    except Exception as e:
        print(f"Error downloading image from {image_url}: {e}")

# URL 순회하면서 이미지 처리
for url in url_list:
    # 페이지 요청
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # view_cont div 찾기
        div_section = soup.find('div', {'class': 'view_cont'})
        
        if div_section:
            # 이미지 태그 찾기
            img_tag = div_section.find('img')
            if img_tag and 'src' in img_tag.attrs:
                image_url = img_tag.attrs['src']
                
                # 상대 URL 처리 (만약 src가 상대경로라면)
                image_url = urljoin(url, image_url)
                image_number = url.split("/")[-1]
                
                # 이미지 저장 경로 (번호 + .jpeg)
                image_path = os.path.join(image_folder, f"{image_number}.jpeg")
                
                
                # 이미지 다운로드
                download_image(image_url, image_path)
            else:
                print("No image found in the view_cont section.")
        else:
            print("No view_cont section found.")
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

Image saved at /Users/lee/Desktop/chatDPT/image/26758371.jpeg
Image saved at /Users/lee/Desktop/chatDPT/image/26758365.jpeg
Image saved at /Users/lee/Desktop/chatDPT/image/26758350.jpeg
Image saved at /Users/lee/Desktop/chatDPT/image/26758345.jpeg
No image found in the view_cont section.
No image found in the view_cont section.
Image saved at /Users/lee/Desktop/chatDPT/image/26758338.jpeg
Image saved at /Users/lee/Desktop/chatDPT/image/26758337.jpeg
Image saved at /Users/lee/Desktop/chatDPT/image/26758332.jpeg
Image saved at /Users/lee/Desktop/chatDPT/image/26758330.jpeg


In [4]:
import os
import requests
from bs4 import BeautifulSoup
import base64
from openai import OpenAI

client = OpenAI()

# 이미지 폴더 경로
image_folder = os.path.join(os.getcwd(), "image")

# GPT-4 텍스트 변환 함수
def analyze_data_with_gpt4(image_path):
    # 올바른 image_path 처리
    image_path = os.path.join(os.getcwd(), "image", image_path)  # {image_path} -> image_path로 수정

    with open(image_path, "rb") as image_file:
        current_base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Turn an image into text with as much content as possible. Don't use special characters, write in lines. and Answer in Korean."},
            {"role": "user", "content": [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{current_base64_image}"}}]}
        ]
    )
    advice = response.choices[0].message.content
    return advice

content = []

# URL 리스트 순회
for url in url_list:
    # 페이지 요청
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # view_cont div 찾기
        div_section = soup.find('div', {'class': 'view_cont'})
        

        if div_section:
            # view_cont 내의 텍스트 추출
            content_text = div_section.get_text(separator='\n', strip=True)
            
            image_name = url.split("/")[-1] + ".jpeg"  # 이미지 파일 이름 생성
            image_path = os.path.join(image_folder, image_name)
            
            # 이미지 파일이 존재하면, GPT 모델로 텍스트 변환
            if os.path.exists(image_path):
                explanation = analyze_data_with_gpt4(image_name)  # 이미지 경로를 함수로 전달
                content_text += explanation  # 이미지 설명 추가
            
            # 텍스트 출력 및 저장
            if content_text:
                content.append(content_text)
            else:
                print(f"No text found in view_cont for {url}.")
        else:
            print(f"view_cont section not found for {url}.")
    else:
        print(f"Failed to retrieve the page {url}. Status code: {response.status_code}")

In [5]:
url_content_mapping = dict(zip(url_list, content))
url_content_mapping

{'https://www.dongguk.edu/article/GENERALNOTICES/detail/26758381': "💻\n동국대학교 DUICA\n에서\n겨울방학 특강\n수강생을 모집합니다!\n\u200b\n겨울방학, 알차게 보내고 싶으신가요?\n배우고 싶었던 교육과정을 이번 기회에 만나보세요!\n1. <프로그래밍 입문·기초반(JAVA)>\n프로그래밍을 처음 학습하고자하는 수강생들을 위한 특강\n❗\n\u200b\n전공, 비전공자 불문, 프로그래밍 입문자를 위한\n눈높이 맞춤교육을 소개합니다.🤗\n\u200b\n입문기초부터 튼튼히!\n향후 프로그래밍 심화 과정 학습이 훨씬 수월해지시겠죠?\n⬇ 수강신청 URL ⬇\nhttps://forms.gle/fGvj4qwySrBBwtNcA\n2. <경영정보시각화능력자격과정> 모집 안내\n요즘 비지니스대세! 데이터 시각화!\n\u200b\n2024년, 올해 처음 도입된 최신 국가공인자격증인\n<경엉정보시각화능력>\n자격증 취득을\n준비할 수 있는 특강을 소개해드려요.😉\n\u200b\n경영 이론과 데이터시각화 실무를 한번에 배울 수 있는 과정으로\n삼성,SK, 현대차 등\n주요 대기업의 채용우대 자격증이라고 하니\n앞으로 더 많은 수요가 있을 것으로 예상되는 자격증입니다.\n⬇ 수강신청 URL ⬇\nhttps://forms.gle/EXz6BEDYP529iRFYA\n****\n겨울방학 동안\n스펙을 쌓고 싶은 분들!\n알차게 한 해를 시작하고 싶은 분들은\n\u200b\n동국대 듀이카\n'경영정보시각화능력자격과정',\n'프로그래밍 입문·기초반(JAVA)'\n\u200b\n특강을 도전해보세요!\n❗신청기한❗\n~ 2025. 01.02.(목)까지",
 'https://www.dongguk.edu/article/GENERALNOTICES/detail/26758371': 'Microsoft MLP(Microsoft Learning Platform) 연계 IT 분야 겨울방학 온라인 특강 안내\n[MLP(Microsoft Learn

In [6]:
import sqlite3

# Initialize SQLite database
def initialize_db(db_path):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS DONGGUK_BOARD (
                url TEXT PRIMARY KEY,
                context TEXT 
            );
        ''')
        conn.commit()

In [7]:
initialize_db('notification.sqlite')

In [8]:
def save_to_db(url, context, db_path='notification.sqlite'):
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            INSERT OR IGNORE INTO DONGGUK_BOARD (url, context) VALUES (?, ?);
        ''', (url, context))
        conn.commit()

for key, value in url_content_mapping.items():
    save_to_db(key, value)

In [9]:
import sqlite3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_upstage import UpstageEmbeddings

def load_data_from_db(db_path='notification.sqlite'):
    """Load data from SQLite database and return a list of (url, context) tuples."""
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT url, context FROM DONGGUK_BOARD")
        rows = cursor.fetchall()
    return rows

def combine_url_with_context(rows):
    """
    Combine each URL with its corresponding context and split into smaller chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)
    combined_documents = []

    for url, context in rows:
        # Combine URL and context
        combined_text = f"URL: {url}\n\n{context}"
        # Split the combined text into smaller chunks
        chunks = text_splitter.split_text(combined_text)
        combined_documents.extend(chunks)

    return combined_documents

def save_to_chroma(combined_documents):
    """
    Save the combined documents (context + URL) into Chroma database with embeddings.
    """
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    database = Chroma(
        collection_name='chroma-dongguk',
        persist_directory="./chroma",
        embedding_function=embeddings
    )
    
    # Generate embeddings and save to Chroma
    new_embeddings = embeddings.embed_documents(combined_documents)
    database.add_texts(texts=combined_documents, embeddings=new_embeddings)

    # Persist the database
    database.persist()

def main():
    db_path = 'notification.sqlite'
    rows = load_data_from_db(db_path)
    combined_docs = combine_url_with_context(rows)
    save_to_chroma(combined_docs)

In [10]:
main()

/var/folders/hk/_c2tkv0s50x4hnyx4hn769780000gn/T/ipykernel_10386/978109356.py:35: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  database = Chroma(
/var/folders/hk/_c2tkv0s50x4hnyx4hn769780000gn/T/ipykernel_10386/978109356.py:46: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  database.persist()
